# Feature Enrichment untuk Dataset KreditLensa

Script ini bertanggung jawab untuk memperkaya dataset utama `KreditLensa_Dataset.csv` dengan data sosio-ekonomi tambahan dari Indonesia, seperti PDRB (Produk Domestik Regional Bruto), TPT (Tingkat Pengangguran Terbuka), dan data UMKM nasional. Proses ini meliputi pemuatan data, pembersihan, rekayasa fitur, penggabungan dataset, dan penyimpanan dataset yang sudah diperkaya.

In [ ]:
import pandas as pd
import numpy as np
import warnings

# Mengabaikan peringatan untuk menjaga output tetap bersih
warnings.filterwarnings('ignore')

## Bagian 1: Pemuatan Dataset

Pada bagian ini, semua dataset yang diperlukan akan dimuat ke dalam DataFrame Pandas.

In [ ]:
print("Memuat dataset...")

# Memuat dataset utama KreditLensa
df_kredit = pd.read_csv(
    '/home/neva/DS_PROJECT/Datathon/Dataset_KreditLensa/KreditLensa_Dataset.csv',
    low_memory=False
)

# Memuat data PDRB (Produk Domestik Regional Bruto) per provinsi
df_pdrb = pd.read_csv(
    '/home/neva/DS_PROJECT/Datathon/Dataset_KreditLensa/[Seri 2010] Distribusi PDRB Terhadap Seluruh Provinsi Indonesia Atas Dasar Harga Berlaku Menurut Provinsi, 2024.csv',
    skiprows=4 # Melewatkan baris header yang tidak relevan
)

# Memuat data posisi kredit UMKM nasional
df_umkm = pd.read_csv(
    '/home/neva/DS_PROJECT/Datathon/Dataset_KreditLensa/Posisi Kredit Usaha Mikro, Kecil, dan Menengah (UMKM) 1 pada Bank Umum (miliar rupiah), 2012-2016.csv',
    skiprows=3 # Melewatkan baris header yang tidak relevan
)

# Memuat data Tingkat Pengangguran Terbuka (TPT) dan Tingkat Partisipasi Angkatan Kerja (TPAK)
df_tpt = pd.read_csv(
    '/home/neva/DS_PROJECT/Datathon/Dataset_KreditLensa/Tingkat Pengangguran Terbuka (TPT) dan Tingkat Partisipasi Angkatan Kerja (TPAK) Menurut Provinsi, 2023.csv'
)

print("Dataset berhasil dimuat.")

## Bagian 2: Pembersihan dan Persiapan Data

Bagian ini fokus pada pembersihan dan transformasi data agar siap untuk proses penggabungan.

In [ ]:
print("Memulai pembersihan dan persiapan data...")

# 2.1. Pembersihan df_kredit (Dataset Utama)
print("  - Membersihkan data kredit...")
# Mengubah kolom mata uang menjadi tipe float
currency_cols = ['DisbursementGross', 'GrAppv', 'SBA_Appv']
for col in currency_cols:
    if df_kredit[col].dtype == 'object':
        df_kredit[col] = df_kredit[col].str.replace('$', '', regex=False).str.replace(',', '', regex=False).astype(float)

# Mengubah kolom tanggal menjadi format datetime dan mengekstrak tahun persetujuan
df_kredit['ApprovalDate'] = pd.to_datetime(df_kredit['ApprovalDate'], errors='coerce')
df_kredit['TahunPersetujuan'] = df_kredit['ApprovalDate'].dt.year

# Mengubah 'MIS_Status' menjadi biner (1 untuk CHGOFF/Gagal Bayar, 0 untuk lainnya)
df_kredit['MIS_Status'] = df_kredit['MIS_Status'].apply(lambda x: 1 if str(x).strip() == 'CHGOFF' else 0)

In [ ]:
# 2.2. Pembersihan df_pdrb
print("  - Membersihkan data PDRB...")
# Mengganti nama kolom pertama menjadi 'Provinsi'
df_pdrb = df_pdrb.rename(columns={df_pdrb.columns[0]: 'Provinsi'})
# Melakukan unpivot data dari format lebar ke panjang
df_pdrb = df_pdrb.melt(id_vars=['Provinsi'], var_name='Tahun', value_name='Distribusi_PDRB')
# Mengubah kolom 'Tahun' menjadi numerik
df_pdrb['Tahun'] = pd.to_numeric(df_pdrb['Tahun'])

In [ ]:
# 2.3. Pembersihan df_tpt
print("  - Membersihkan data TPT...")
# Mengubah kolom TPT menjadi numerik, mengubah nilai bermasalah menjadi NaN
for col in ['Tingkat Pengangguran Terbuka (TPT) - Februari', 'Tingkat Pengangguran Terbuka (TPT) - Agustus']:
    df_tpt[col] = pd.to_numeric(df_tpt[col], errors='coerce')
# Menghapus baris dengan NaN di kolom TPT yang relevan
df_tpt.dropna(subset=['Tingkat Pengangguran Terbuka (TPT) - Februari', 'Tingkat Pengangguran Terbuka (TPT) - Agustus'], inplace=True)
# Menghitung rata-rata TPT tahunan dari data Februari dan Agustus
df_tpt['TPT_Tahunan'] = df_tpt[['Tingkat Pengangguran Terbuka (TPT) - Februari', 'Tingkat Pengangguran Terbuka (TPT) - Agustus']].mean(axis=1)
# Memilih kolom yang relevan dan menambahkan kolom 'Tahun'
df_tpt = df_tpt[['Provinsi', 'TPT_Tahunan']].copy()
df_tpt['Tahun'] = 2023 # Data TPT yang tersedia adalah untuk tahun 2023

In [ ]:
# 2.4. Pembersihan df_umkm
print("  - Membersihkan data UMKM...")
# Mengganti nama kolom untuk kemudahan akses
df_umkm.columns = ['Rincian', '2012', '2013', '2014', '2015', '2016']
# Memilih baris yang relevan (Total Modal Kerja) dan melakukan unpivot
df_umkm_total = df_umkm[df_umkm['Rincian'] == 'Modal Kerja'].copy()
df_umkm_total = df_umkm_total.melt(id_vars=['Rincian'], var_name='Tahun', value_name='Total_Kredit_UMKM_Nasional')
# Mengubah kolom 'Tahun' menjadi numerik
df_umkm_total['Tahun'] = pd.to_numeric(df_umkm_total['Tahun'])
# Membersihkan dan mengubah kolom nilai kredit menjadi float
df_umkm_total['Total_Kredit_UMKM_Nasional'] = df_umkm_total['Total_Kredit_UMKM_Nasional'].str.replace(' ', '').astype(float)
# Memilih kolom yang relevan
df_umkm_total = df_umkm_total[['Tahun', 'Total_Kredit_UMKM_Nasional']]

print("Pembersihan dan persiapan data selesai.")

## Bagian 3: Rekayasa Fitur (Feature Engineering)

Pada bagian ini, dilakukan rekayasa fitur, termasuk pemetaan negara bagian AS ke provinsi Indonesia untuk tujuan simulasi.

In [ ]:
print("Memulai rekayasa fitur...")

# 3.1. Pemetaan Negara Bagian AS ke Provinsi Indonesia (Simulasi)
# Karena dataset asli menggunakan negara bagian AS, kita akan memetakannya secara acak
# ke provinsi-provinsi di Indonesia untuk tujuan simulasi dan enrichment.
print("  - Memetakan negara bagian AS ke provinsi Indonesia (simulasi)...")
us_states = df_kredit['State'].unique()
indonesian_provinces = df_pdrb['Provinsi'].dropna().unique()

# Menetapkan seed untuk reproduktifitas, sehingga pemetaan 'acak' selalu sama.
np.random.seed(42)

# Membuat mapping acak. Dalam skenario nyata, ini akan didasarkan pada data geografis/demografis yang relevan.
state_to_province_map = {state: province for state, province in zip(us_states, np.random.choice(indonesian_provinces, len(us_states), replace=True))}
df_kredit['Provinsi'] = df_kredit['State'].map(state_to_province_map)

print("Rekayasa fitur selesai.")

## Bagian 4: Penggabungan Dataset (Feature Enrichment)

Dataset-dataset yang telah dibersihkan dan disiapkan akan digabungkan untuk memperkaya dataset utama.

In [ ]:
print("Menggabungkan dataset...")
    
# 4.1. Menggabungkan semua dataset dengan metode 'left'
df_enriched = pd.merge(df_kredit, df_pdrb, left_on=['TahunPersetujuan', 'Provinsi'], right_on=['Tahun', 'Provinsi'], how='left')
df_enriched = pd.merge(df_enriched, df_tpt, on=['TahunPersetujuan', 'Provinsi'], how='left')
df_enriched = pd.merge(df_enriched, df_umkm_total, left_on='TahunPersetujuan', right_on='Tahun', how='left')
    
# 4.2. Logika Pengisian Cerdas untuk Data Ekonomi
print("  - Mengisi data ekonomi yang kosong menggunakan metode forward/backward fill...")
# Mengisi data per provinsi untuk TPT dan PDRB
df_enriched['TPT_Tahunan'] = df_enriched.groupby('Provinsi')['TPT_Tahunan'].transform(lambda x: x.ffill().bfill())
df_enriched['Distribusi_PDRB'] = df_enriched.groupby('Provinsi')['Distribusi_PDRB'].transform(lambda x: x.ffill().bfill())
# Mengisi data nasional untuk kredit UMKM
df_enriched['Total_Kredit_UMKM_Nasional'] = df_enriched['Total_Kredit_UMKM_Nasional'].ffill().bfill()
    
# 4.3. Membersihkan kolom tahun yang duplikat
cols_to_drop = [col for col in df_enriched.columns if 'Tahun_x' in str(col) or 'Tahun_y' in str(col) or 'Tahun' == str(col)]
df_enriched = df_enriched.drop(columns=cols_to_drop)
    
print("Penggabungan dataset selesai.")

## Bagian 5: Penggantian Nama Kolom (Opsional, untuk kejelasan)

Kolom-kolom dalam dataset yang diperkaya akan diganti namanya untuk kejelasan dan konsistensi.

In [ ]:
print("Mengganti nama kolom untuk kejelasan...")
column_mapping = {
    'Name': 'Nama_UMKM',
    'City': 'Kota',
    'State': 'Negara_Bagian_AS',
    'Zip': 'Kode_Pos',
    'Bank': 'Nama_Bank',
    'BankState': 'Negara_Bagian_Bank',
    'NAICS': 'Kode_Industri_NAICS',
    'ApprovalDate': 'Tanggal_Persetujuan',
    'ApprovalFY': 'Tahun_Fiskal_Persetujuan',
    'Term': 'Jangka_Waktu_Bulan',
    'NoEmp': 'Jumlah_Karyawan',
    'NewExist': 'Status_Usaha_Baru',
    'CreateJob': 'Penciptaan_Lapangan_Kerja',
    'RetainedJob': 'Pekerjaan_Yang_Dipertahankan',
    'FranchiseCode': 'Kode_Waralaba',
    'UrbanRural': 'Status_Lokasi_Urban_Rural',
    'RevLineCr': 'Kredit_Revolving',
    'LowDoc': 'Dokumen_Ringkas',
    'ChgOffDate': 'Tanggal_Penghapusan_Kredit',
    'DisbursementDate': 'Tanggal_Pencairan',
    'DisbursementGross': 'Jumlah_Pencairan_Kotor',
    'BalanceGross': 'Saldo_Kotor',
    'MIS_Status': 'Status_Kredit_Macet',
    'ChgOffPrinGr': 'Jumlah_Penghapusan_Pokok',
    'GrAppv': 'Jumlah_Pinjaman_Disetujui',
    'SBA_Appv': 'Jumlah_Jaminan_SBA',
    'TahunPersetujuan': 'Tahun_Persetujuan',
    'Provinsi': 'Provinsi',
    'Distribusi_PDRB': 'Distribusi_PDRB_Persen',
    'TPT_Tahunan': 'TPT_Tahunan_Persen',
    'Total_Kredit_UMKM_Nasional': 'Total_Kredit_UMKM_Nasional_Miliar'
}
df_enriched.rename(columns=column_mapping, inplace=True)
print("Penggantian nama kolom selesai.")

## Bagian 6: Penyimpanan Dataset yang Diperkaya

Dataset yang telah diperkaya akan disimpan ke dalam file CSV baru.

In [ ]:
print("Menyimpan dataset yang diperkaya...")
df_enriched.to_csv(
    '/home/neva/DS_PROJECT/Datathon/Dataset_KreditLensa/Dataset_Kredit_Diperkaya.csv',
    index=False
)
print("Proses feature enrichment selesai. Dataset baru disimpan sebagai 'Dataset_Kredit_Diperkaya.csv'")